# Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv


# Dataset Loading

In [29]:
train = pd.read_csv(r'D:\Project Phase II\Dataset\finaltrain.csv',encoding='cp1252')
test = pd.read_csv(r'D:\Project Phase II\Dataset\finaltest.csv',encoding='cp1252') #test_set1 
autest = pd.read_csv(r'D:\Project Phase II\Dataset\UNSW_NB15_testing-set.csv',encoding='cp1252') #test_set2

# Data Preprocessing

## Preprocess user-made train-test datasets

In [30]:
todrop = ['sloss', 'dloss','stcpb', 'dtcpb', 'trans_depth', 'Stime', 'Ltime','ct_flw_http_mthd', 
        'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm',
        'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat']
        
reducedTrain = train.drop(todrop, axis = 1)
reducedTest = test.drop(todrop, axis = 1)
reducedTrain = reducedTrain.drop_duplicates()
reducedTest = reducedTest.drop_duplicates()

trainAttributes = reducedTrain.drop(['srcip','sport','dstip','dsport','Label'], axis = 1)
trainLabel = reducedTrain['Label']
testAttributes = reducedTest.drop(['srcip','sport','dstip','dsport','Label'], axis = 1)
testLabel = reducedTest['Label']

train = reducedTrain
test = reducedTest

train = train.drop_duplicates(['srcip','sport', 'dstip', 'dsport'], keep = 'last')
test = test.drop_duplicates(['srcip','sport', 'dstip', 'dsport'], keep = 'last')

In [7]:
print("Train Set:\n",train.Label.value_counts())
print("Test Set:\n", test.Label.value_counts())

Train Set:
 Label
0    39990
1    39821
Name: count, dtype: int64
Test Set:
 Label
1    17217
0    17201
Name: count, dtype: int64


## Preprocess Author's testing dataset

In [31]:
autestdrop = ['ï»¿id', 'rate', 'sloss', 'dloss','stcpb', 'dtcpb','trans_depth','ct_srv_src','ct_dst_ltm',
        'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
        'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
        'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat']

autest = autest.drop(autestdrop, axis = 1)
#Change column names of autest
autest.rename(columns={'dinpkt':'Dintpkt','djit':'Djit','dload':'Dload','dpkts':'Dpkts','label':'Label','sinpkt':'Sintpkt','sjit':'Sjit', 'sload':'Sload', 'spkts':'Spkts','dmean':'dmeansz','response_body_len':'res_bdy_len', 'smean':'smeansz'}, inplace=True)
autest = autest[(autest['proto']=='tcp') | (autest['proto']=='udp') | (autest['proto']=='ospf')]
autest = autest[(autest['service']=='ssh') | (autest['service']=='ftp-data ')  | (autest['service']=='ftp') | (autest['service']=='-') | (autest['service']=='dns') | (autest['service']=='smtp') | (autest['service']=='http') | (autest['service']=='radius') | (autest['service']=='pop3') ]
autest = autest[(autest['state'] == 'CON') | (autest['state'] == 'RST') | (autest['state'] == 'FIN') | (autest['state'] == 'ACC') | (autest['state'] == 'REQ') | (autest['state'] == 'INT')]

autest['proto'].replace("tcp", 0, inplace = True)
autest['proto'].replace("udp", 1, inplace = True)
autest['proto'].replace("ospf", 2, inplace = True)

autest['service'].replace("ssh", 0, inplace = True)
autest['service'].replace("ftp-data", 1, inplace = True)
autest['service'].replace("ftp", 2, inplace = True)
autest['service'].replace("-", 3, inplace = True)
autest['service'].replace("dns", 4, inplace = True)
autest['service'].replace("smtp", 5, inplace = True)
autest['service'].replace("http", 6, inplace = True)
autest['service'].replace("radius", 7, inplace = True)
autest['service'].replace("pop3", 8, inplace = True)

autest['state'].replace("CON", 0, inplace = True)
autest['state'].replace("RST", 1, inplace = True)
autest['state'].replace("FIN", 2, inplace = True)
autest['state'].replace("ACC", 3, inplace = True)
autest['state'].replace("REQ", 4, inplace = True)
autest['state'].replace("INT", 5, inplace = True)

for column in autest.columns:
        if column != 'Label':
                col_mean = sum(autest[column]) / len(autest[column])
                col_std = (sum((x - col_mean) ** 2 for x in autest[column]) / len(autest[column])) ** 0.5
                autest[column] = [(x - col_mean) / col_std for x in autest[column]]


autest = autest.drop_duplicates()
autestAttributes = autest.drop(['Label'], axis = 1)
autestAttributes = autestAttributes[trainAttributes.columns]
autestLabel = autest['Label']

In [59]:
print("Author's Testing Set:\n", autest.Label.value_counts())

Author's Testing Set:
 Label
0    29673
1    14973
Name: count, dtype: int64


# Input Transformation

## Training

In [33]:
nodes = set()
trainUnique = train
for i in range(len(train)):
    src = str(train['srcip'].iloc[i])+':'+str(train['sport'].iloc[i])
    dst = str(train['dstip'].iloc[i])+':'+str(train['dsport'].iloc[i])
    nodes.add(src)
    nodes.add(dst)


#Find unique (srcip,sport,dstip,dsport) tuples in train
train_tuples = set()
for i in range(len(train)):
    train_tuples.add((train['srcip'].iloc[i], train['sport'].iloc[i], train['dstip'].iloc[i], train['dsport'].iloc[i]))

In [34]:
xLookUp = {}
x = []
count = 0
edge_index = []
edge_attr = []
for i in range(len(train)):
    src = str(train['srcip'].iloc[i])+':'+str(train['sport'].iloc[i])
    dst = str(train['dstip'].iloc[i])+':'+str(train['dsport'].iloc[i])
    if src not in xLookUp:
        xLookUp[src] = count
        x.append([int(train['sport'].iloc[i])])
        count += 1
    if dst not in xLookUp:
        xLookUp[dst] = count
        x.append([int(train['dsport'].iloc[i])])
        count += 1
    edge_index.append([xLookUp[src], xLookUp[dst]])
    edge_attr.append(list(train.iloc[i].drop(['srcip', 'sport', 'dstip', 'dsport','Label']).values))


# Convert x to tensor
x_tensor = torch.tensor(x, dtype=torch.float)

# Convert edge_index to tensor
edge_index_tensor = torch.tensor(edge_index, dtype=torch.long).t()  # Transpose for correct format

# Convert edge_attr to tensor
edge_attr_tensor = torch.tensor(edge_attr, dtype=torch.float)

print(x_tensor.shape, edge_index_tensor.shape, edge_attr_tensor.shape)

torch.Size([94760, 1]) torch.Size([2, 79811]) torch.Size([79811, 25])


## Testing

In [8]:
testnodes = set()
testUnique = test
for i in range(len(test)):
    src = str(test['srcip'].iloc[i])+':'+str(test['sport'].iloc[i])
    dst = str(test['dstip'].iloc[i])+':'+str(test['dsport'].iloc[i])
    testnodes.add(src)
    testnodes.add(dst)


#Find unique (srcip,sport,dstip,dsport) tuples in train
test_tuples = set()
for i in range(len(test)):
    test_tuples.add((test['srcip'].iloc[i], test['sport'].iloc[i], test['dstip'].iloc[i], test['dsport'].iloc[i]))

In [9]:
xtestLookUp = {}
xtest = []
count = 0
edge_index_test = []
edge_attr_test = []
for i in range(len(test)):
    src = str(test['srcip'].iloc[i])+':'+str(test['sport'].iloc[i])
    dst = str(test['dstip'].iloc[i])+':'+str(test['dsport'].iloc[i])
    if src not in xtestLookUp:
        xtestLookUp[src] = count
        xtest.append([int(test['sport'].iloc[i])])
        count += 1
    if dst not in xtestLookUp:
        xtestLookUp[dst] = count
        xtest.append([int(test['dsport'].iloc[i])])
        count += 1
    edge_index_test.append([xtestLookUp[src], xtestLookUp[dst]])
    edge_attr_test.append(list(test.iloc[i].drop(['srcip', 'sport', 'dstip', 'dsport','Label']).values))


# Convert x to tensor
x_test_tensor = torch.tensor(xtest, dtype=torch.float)

# Convert edge_index to tensor
edge_index_test_tensor = torch.tensor(edge_index_test, dtype=torch.long).t()  # Transpose for correct format

# Convert edge_attr to tensor
edge_attr_test_tensor = torch.tensor(edge_attr_test, dtype=torch.float)

print(x_test_tensor.shape, edge_index_test_tensor.shape, edge_attr_test_tensor.shape)

torch.Size([42412, 1]) torch.Size([2, 34418]) torch.Size([34418, 25])


## Author's Testing Dataset

Since srcip, sport, dstip, dsport information is not available, we proceed with having each node feature value as 0, and assume each edge as an unique connection between two imaginary nodes

In [32]:
n_autestnodes = int(len(autest) ** 0.5)
n_autest_tuples = n_autestnodes ** 2

xautest = [[0] for j in range(n_autest_tuples)]
edge_index_autest = []
edge_attr_autest = []
auEdgeLabel = []
count = 0
for i in range(n_autestnodes):
    for j in range(n_autestnodes):
        if(i == count):
            count += 1
            continue
        edge_index_autest.append([i, count])
        edge_attr_autest.append(list(autest.iloc[count].drop(['Label']).values))
        count += 1
        auEdgeLabel.append(autest.iloc[count].Label)
        

# Convert x to tensor
x_autest_tensor = torch.tensor(xautest, dtype=torch.float)

# Convert edge_index to tensor
edge_index_autest_tensor = torch.tensor(edge_index_autest, dtype=torch.long).t()  # Transpose for correct format

# Convert edge_attr to tensor
edge_attr_autest_tensor = torch.tensor(edge_attr_autest, dtype=torch.float)

print(x_autest_tensor.shape, edge_index_autest_tensor.shape, edge_attr_autest_tensor.shape)

torch.Size([44521, 1]) torch.Size([2, 44520]) torch.Size([44520, 25])


# GAT Model

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GATClassifier(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features, hidden_channels, num_heads):
        super(GATClassifier, self).__init__()
        self.conv1 = GATConv(num_node_features, hidden_channels, heads=num_heads, add_self_loops=False)
        self.conv2 = GATConv(hidden_channels * num_heads, hidden_channels, heads=num_heads, add_self_loops=False)
        # self.edge_classifier = nn.Linear(hidden_channels * num_heads + num_edge_features, 1)  # Output size changed to 1
        self.edge_classifier = nn.Linear(hidden_channels * 2 * num_heads + num_edge_features, 1)  # Output size changed to 1

    def forward(self, x, edge_index, edge_attr):
        # Apply graph attentional layers
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)

        # Concatenate node features with edge features
        num_edges = edge_index.size(1)
        node_x = x[edge_index[0]]
        node_x_other = x[edge_index[1]]
        edge_attr_padded = torch.cat([edge_attr, torch.zeros(num_edges - edge_attr.size(0), edge_attr.size(1))], dim=0)
        edge_x = torch.cat([node_x, node_x_other, edge_attr_padded], dim=1)
        print("edge_x.shape:", edge_x.shape)
        print("self.edge_classifier.weight.shape:",self.edge_classifier.weight.shape)
        # Classify edges into binary class using BCELogit
        edge_scores = self.edge_classifier(edge_x)
        return edge_scores


Training

In [36]:
# Example usage
num_nodes = len(nodes)
num_edges = len(train_tuples)
num_node_features = 1
num_edge_features = 25
hidden_channels = 32
num_heads = 8

# Instantiate GAT classifier
model = GATClassifier(num_node_features, num_edge_features, hidden_channels, num_heads)

# Forward pass
edge_scores_train = model(x_tensor, edge_index_tensor, edge_attr_tensor)

edge_x.shape: torch.Size([79811, 537])
self.edge_classifier.weight.shape: torch.Size([1, 537])


Testing

In [21]:
# Example usage
num_nodes_test = len(testnodes)
num_edges_test = len(test_tuples)
num_node_features = 1
num_edge_features = 25
hidden_channels = 32
num_heads = 8

# Forward pass
edge_scores_test = model(x_test_tensor, edge_index_test_tensor, edge_attr_test_tensor)

edge_x.shape: torch.Size([34418, 537])
self.edge_classifier.weight.shape: torch.Size([1, 537])


Author's testing dataset

In [37]:
# Example usage
num_nodes_test = n_autestnodes
num_edges_test = n_autest_tuples
num_node_features = 1
num_edge_features = 25
hidden_channels = 32
num_heads = 8

# Forward pass
edge_scores_autest = model(x_autest_tensor, edge_index_autest_tensor, edge_attr_autest_tensor)

edge_x.shape: torch.Size([44520, 537])
self.edge_classifier.weight.shape: torch.Size([1, 537])


# KNN

In [22]:
import pandas as pd
# Convert edge_scores tensor to a NumPy array
edge_scores_train_array = edge_scores_train.detach().numpy()
edge_scores_test_array = edge_scores_test.detach().numpy()
# Create a DataFrame from the NumPy array
edge_scores_train_df = pd.DataFrame(edge_scores_train_array, columns=['Edge_Scores'])
edge_scores_test_df = pd.DataFrame(edge_scores_test_array, columns=['Edge_Scores'])
trainLabel = train['Label']
testLabel = test['Label']

In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

X_train = edge_scores_train_df
X_test = edge_scores_test_df
y_train = trainLabel
y_test = testLabel

# Initialize the KNN classifier
k = 15  # Number of neighbors
knn_classifier = KNeighborsClassifier(n_neighbors=k)

# Train the classifier
knn_classifier.fit(X_train, y_train)

# Predictions on the test set
y_pred = knn_classifier.predict(X_test)

# Calculate accuracy
report = classification_report(y_test, y_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

           0       0.93      0.96      0.95     17201
           1       0.96      0.93      0.95     17217

    accuracy                           0.95     34418
   macro avg       0.95      0.95      0.95     34418
weighted avg       0.95      0.95      0.95     34418



Author's test dataset

In [38]:
import pandas as pd
# Convert edge_scores tensor to a NumPy array
edge_scores_autest_array = edge_scores_autest.detach().numpy()
# Create a DataFrame from the NumPy array
testLabeldf = pd.DataFrame(auEdgeLabel, columns=['Label'])
edge_scores_autest_df = pd.DataFrame(edge_scores_autest_array, columns=['Edge_Scores'])
trainLabel = train['Label']
autestLabel = testLabeldf['Label']
auX_test = edge_scores_autest_df
auy_test = autestLabel 

auy_pred = knn_classifier.predict(edge_scores_autest_df)

# Calculate accuracy
report = classification_report(autestLabel, auy_pred)
print("Classification Report: \n", report)

Classification Report: 
               precision    recall  f1-score   support

         0.0       0.66      0.56      0.61     29547
         1.0       0.33      0.42      0.37     14973

    accuracy                           0.51     44520
   macro avg       0.49      0.49      0.49     44520
weighted avg       0.55      0.51      0.53     44520



# Lazy Predict Scores

In [39]:
# List of classifiers to include
import sklearn
import xgboost
import lightgbm
import sklearn.discriminant_analysis
from lazypredict.Supervised import LazyClassifier

#Skip SVM
classifiers = [
 ('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
 ('BernoulliNB', sklearn.naive_bayes.BernoulliNB),
 ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
 ('DummyClassifier', sklearn.dummy.DummyClassifier),
 ('GaussianNB', sklearn.naive_bayes.GaussianNB),
 ('KNeighborsClassifier',  sklearn.neighbors._classification.KNeighborsClassifier),
 ('LinearDiscriminantAnalysis',  sklearn.discriminant_analysis.LinearDiscriminantAnalysis),
 ('LogisticRegression', sklearn.linear_model._logistic.LogisticRegression),
 ('Perceptron', sklearn.linear_model._perceptron.Perceptron),
 ('QuadraticDiscriminantAnalysis',  sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis),
 ('RandomForestClassifier', sklearn.ensemble._forest.RandomForestClassifier),
 ('StackingClassifier', sklearn.ensemble._stacking.StackingClassifier),
 ('XGBClassifier', xgboost.sklearn.XGBClassifier),
 ('LGBMClassifier', lightgbm.sklearn.LGBMClassifier)]
clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None,classifiers=classifiers,predictions=True)

models,predictions = clf.fit(X_train, X_test, y_train, y_test)


'tuple' object has no attribute '__name__'
Invalid Classifier(s)


  7%|▋         | 1/15 [00:02<00:30,  2.16s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.9453483642280203, 'Balanced Accuracy': 0.9453537685986607, 'ROC AUC': 0.9453537685986607, 'F1 Score': 0.945341240762079, 'Time taken': 2.161346435546875}


 13%|█▎        | 2/15 [00:04<00:26,  2.06s/it]

{'Model': 'BaggingClassifier', 'Accuracy': 0.9275669707711082, 'Balanced Accuracy': 0.9275716149246278, 'ROC AUC': 0.9275716149246278, 'F1 Score': 0.9275600298027334, 'Time taken': 1.9940049648284912}
{'Model': 'BernoulliNB', 'Accuracy': 0.4973269800685688, 'Balanced Accuracy': 0.497096163727269, 'ROC AUC': 0.497096163727269, 'F1 Score': 0.33280947513195097, 'Time taken': 0.04751086235046387}


...

{'Model': 'RandomForestClassifier', 'Accuracy': 0.9272183159974432, 'Balanced Accuracy': 0.9272221496744331, 'ROC AUC': 0.927222149674433, 'F1 Score': 0.9272136054160849, 'Time taken': 20.186728477478027}


 93%|█████████▎| 14/15 [00:26<00:02,  2.74s/it]

{'Model': 'XGBClassifier', 'Accuracy': 0.9417746527979546, 'Balanced Accuracy': 0.941783068387355, 'ROC AUC': 0.941783068387355, 'F1 Score': 0.9417559991171154, 'Time taken': 0.3136274814605713}
[LightGBM] [Info] Number of positive: 39821, number of negative: 39990
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000868 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 79811, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498941 -> initscore=-0.004235
[LightGBM] [Info] Start training from score -0.004235


100%|██████████| 15/15 [00:27<00:00,  1.83s/it]

{'Model': 'LGBMClassifier', 'Accuracy': 0.9424719623452844, 'Balanced Accuracy': 0.9424852404924772, 'ROC AUC': 0.9424852404924772, 'F1 Score': 0.9424256685890511, 'Time taken': 0.5841882228851318}


In [40]:
from sklearn.metrics import roc_auc_score
for model in predictions.columns:
    print(model)
    print(classification_report(y_test, predictions[model]))
    auc_scores = roc_auc_score(y_test, predictions[model], multi_class='ovr')  # Or multi_class='ovo' for one-vs-one
    print("AUC Scores for each class:", auc_scores)
    print('-----------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------')

AdaBoostClassifier
              precision    recall  f1-score   support

           0       0.94      0.96      0.95     17201
           1       0.96      0.93      0.94     17217

    accuracy                           0.95     34418
   macro avg       0.95      0.95      0.95     34418
weighted avg       0.95      0.95      0.95     34418

AUC Scores for each class: 0.9453537685986607
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
BaggingClassifier
              precision    recall  f1-score   support

           0       0.92      0.94      0.93     17201
           1       0.94      0.92      0.93     17217

    accuracy                           0.93     34418
   macro avg       0.93      0.93      0.93     34418
weighted avg       0.93      0.93      0.93     34418

AUC Scores for each class: 0.9275716149246278
-------------------------------------------------

Lazy Predict for author's testing dataset

In [41]:
# List of classifiers to include
import sklearn
import xgboost
import lightgbm
import sklearn.discriminant_analysis
from lazypredict.Supervised import LazyClassifier

#Skip SVM
classifiers = [
 ('AdaBoostClassifier', sklearn.ensemble._weight_boosting.AdaBoostClassifier),
 ('BaggingClassifier', sklearn.ensemble._bagging.BaggingClassifier),
 ('BernoulliNB', sklearn.naive_bayes.BernoulliNB),
 ('DecisionTreeClassifier', sklearn.tree._classes.DecisionTreeClassifier),
 ('DummyClassifier', sklearn.dummy.DummyClassifier),
 ('GaussianNB', sklearn.naive_bayes.GaussianNB),
 ('KNeighborsClassifier',  sklearn.neighbors._classification.KNeighborsClassifier),
 ('LinearDiscriminantAnalysis',  sklearn.discriminant_analysis.LinearDiscriminantAnalysis),
 ('LogisticRegression', sklearn.linear_model._logistic.LogisticRegression),
 ('Perceptron', sklearn.linear_model._perceptron.Perceptron),
 ('QuadraticDiscriminantAnalysis',  sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis),
 ('RandomForestClassifier', sklearn.ensemble._forest.RandomForestClassifier),
 ('StackingClassifier', sklearn.ensemble._stacking.StackingClassifier),
 ('XGBClassifier', xgboost.sklearn.XGBClassifier),
 ('LGBMClassifier', lightgbm.sklearn.LGBMClassifier)]
clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None,classifiers=classifiers,predictions=True)

aumodels,aupredictions = clf.fit(X_train, auX_test, y_train, auy_test)

'tuple' object has no attribute '__name__'
Invalid Classifier(s)


  0%|          | 0/15 [00:00<?, ?it/s]

  7%|▋         | 1/15 [00:02<00:30,  2.16s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.5466307277628032, 'Balanced Accuracy': 0.4898427688674013, 'ROC AUC': 0.4898427688674013, 'F1 Score': 0.54552887099104, 'Time taken': 2.164259195327759}


 13%|█▎        | 2/15 [00:04<00:27,  2.15s/it]

...

{'Model': 'RandomForestClassifier', 'Accuracy': 0.5092542677448337, 'Balanced Accuracy': 0.49321021382672653, 'ROC AUC': 0.4932102138267266, 'F1 Score': 0.5218870757953966, 'Time taken': 19.750532150268555}


 93%|█████████▎| 14/15 [00:27<00:02,  2.93s/it]

{'Model': 'XGBClassifier', 'Accuracy': 0.5467879604672058, 'Balanced Accuracy': 0.4888411792256624, 'ROC AUC': 0.4888411792256623, 'F1 Score': 0.5451196667408759, 'Time taken': 0.4517934322357178}
[LightGBM] [Info] Number of positive: 39821, number of negative: 39990
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000662 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 255
[LightGBM] [Info] Number of data points in the train set: 79811, number of used features: 1
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498941 -> initscore=-0.004235
[LightGBM] [Info] Start training from score -0.004235


100%|██████████| 15/15 [00:28<00:00,  1.87s/it]

{'Model': 'LGBMClassifier', 'Accuracy': 0.5612533692722372, 'Balanced Accuracy': 0.48626558841213874, 'ROC AUC': 0.48626558841213874, 'F1 Score': 0.5490392626873836, 'Time taken': 0.5066995620727539}


In [42]:
from sklearn.metrics import roc_auc_score
for model in aupredictions.columns:
    print(model)
    print(classification_report(auy_test, aupredictions[model]))
    auc_scores = roc_auc_score(auy_test, aupredictions[model], multi_class='ovr')  # Or multi_class='ovo' for one-vs-one
    print("AUC Scores for each class:", auc_scores)
    print('-----------------------------------------------------------------------------------')
    print('-----------------------------------------------------------------------------------')

AdaBoostClassifier
              precision    recall  f1-score   support

         0.0       0.66      0.66      0.66     29547
         1.0       0.32      0.32      0.32     14973

    accuracy                           0.55     44520
   macro avg       0.49      0.49      0.49     44520
weighted avg       0.54      0.55      0.55     44520

AUC Scores for each class: 0.4898427688674013
-----------------------------------------------------------------------------------
-----------------------------------------------------------------------------------
BaggingClassifier
              precision    recall  f1-score   support

         0.0       0.66      0.55      0.60     29547
         1.0       0.33      0.43      0.37     14973

    accuracy                           0.51     44520
   macro avg       0.49      0.49      0.49     44520
weighted avg       0.55      0.51      0.52     44520

AUC Scores for each class: 0.491396999792709
--------------------------------------------------